In [5]:
import pygame
import pygame_menu
import numpy as np

In [76]:
# class Vec1D(np.ndarray):
#     def __new__(cls, magnitude):
#         if isinstance(magnitude, float):
#             return np.asarray(magnitude).view(cls)
#         raise Exception('Vec2D must be 1 dimensional.')
    
class Vec2D(np.ndarray):
    def __new__(cls, iterable):
        if len(iterable) != 2:
            raise Exception('Vec2D must be 2 dimensional.')
        vec = np.asarray(iterable).view(cls)
        return vec
        
    def x(self):
        return self[0]
    
    def y(self):
        return self[1]
    
a, b = Vec2D([2, 1]), Vec2D([2, 4])
c = 2*a - b/2
c

Vec2D([3., 0.])

In [77]:
class Position(Vec2D):
    def distance(self, point):
        return numpy.linalg.norm(self, point)
    
    def xdistance(self, point):
        return abs(self.x() - point.x())
    
    def ydistance(self, point):
        return abs(self.y() - point.y())
    
    def update(self, timestep, velocity, acceleration=Acceleration([0, 0])):
        return self + acceleration/2 * (timestep**2) + velocity * timestep

class Velocity(Vec2D):
    def update(self, timestep, acceleration):
        return self + timestep * acceleration

class Acceleration(Vec2D):
    pass

class Viewport:
    def __init__(self, low, high):
        self.low = low
        self.high = high

In [75]:
# constants

GRAVITY = Acceleration((0, -9.807))
ORIGIN = Position((0, 0))
NO_VELOCITY = Velocity((0, 0))
NO_ACCELERATION = Acceleration((0, 0))

# x = -10m to +10m, y = 0m to +20m
DEFAULT_VIEWPORT = Viewport(Position((-10, 0)), Position((10, 20)))

In [74]:
acc = GRAVITY
vel = Velocity((0, 0))
pos = Position((0, 380))

# it takes just under 9 seconds to fall from the empire state building
for i in range(10):
    print(f't={i}\tpos={pos}\t vel={vel}')
    pos = pos.update(1, vel, acc)
    vel = vel.update(1, acc)

t=0	pos=[  0 380]	 vel=[0 0]
t=1	pos=[  0.     375.0965]	 vel=[ 0.    -9.807]
t=2	pos=[  0.    360.386]	 vel=[  0.    -19.614]
t=3	pos=[  0.     335.8685]	 vel=[  0.    -29.421]
t=4	pos=[  0.    301.544]	 vel=[  0.    -39.228]
t=5	pos=[  0.     257.4125]	 vel=[  0.    -49.035]
t=6	pos=[  0.    203.474]	 vel=[  0.    -58.842]
t=7	pos=[  0.     139.7285]	 vel=[  0.    -68.649]
t=8	pos=[ 0.    66.176]	 vel=[  0.    -78.456]
t=9	pos=[  0.     -17.1835]	 vel=[  0.    -88.263]


In [70]:
class GameObject:
    def __init__(self):
        pass
    
    def update(self):
        raise NotImplementedError('Must override abstract method.')

class PhysicsObject(GameObject):
    def __init__(self, mass, position, velocity=NO_VELOCITY, gravity=GRAVITY, bounciness=1):
        self.mass = mass
        self.position = position
        self.velocity = velocity

    def update(self, timestep):
        pass

    def onCollision(self, gameobject):
        raise NotImplementedError('Must override abstract method.')
        
class PhysicsObject(GameObject):
    def __init__(self, mass, center, radius, velocity=NO_VELOCITY, gravity=GRAVITY, bounciness=1):
        super().__init__(mass, center, velocity=velocity, gravity=gravity)
        self.radius = radius

In [ ]:
class Game:
    def __init__(self, objects):
        self.objects = objects
        
    def update(self, timestep):
        for gameobject in self.objects:
            gameobject.update()

In [2]:
def start(window, clock=pygame.time.Clock(), fps=60):
    width, height = window.get_size()
    
    running = True
    while running:
        # draw current frame
        window.fill((255, 255, 255))
        lizard_rect = pygame.Rect((width / 2 - 10, height / 2 - 10), (100, 100))
        pygame.draw.rect(window, (128, 128, 128), lizard_rect)
        
        # update
        pygame.display.update()
        clock.tick(fps)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_q:
                    running = False


In [3]:
def main_menu(window):
    
    def start_game():
        clock = pygame.time.Clock()
        FPS = 60
        start(window, clock=clock, fps=FPS)
    
    menu = pygame_menu.Menu('Physics Simulator', 400, 400,
                            theme=pygame_menu.themes.THEME_DARK)
    menu.add.button('Play', start_game)
    menu.add.button('Quit', pygame_menu.events.EXIT)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            return

    menu.mainloop(window)

    # menu.add.button('Level Select', load_peg_file_menu, False)
    # menu.add.button('Create', create_level, [])
    # menu.add.button('Full Screen', full_screen_toggle)

In [4]:
def main():
    # pygame initialization
    print('Starting pygame window...')
    pygame.init()
    window = pygame.display.set_mode((0, 0), pygame.FULLSCREEN) # window size 0,0 for full screen
    pygame.display.set_caption("Physics Simulator")

    print('Starting main menu...')
    main_menu(window)

In [5]:
main()

Starting pygame window...
Starting main menu...
